In [1]:
"""
9-29-2018, version 2:
Maybe...we may need to do the feature engineering over here...
Let me save this one as v1, and modify on v2.

9-28-2018, version 1:

Attempt to split the dataset into training and evaluation by using panda.
Given the size of data, I'm a bit worried about the time consumption.

Great reference:
1. https://www.kaggle.com/szelee/how-to-import-a-csv-file-of-55-million-rows

"""

"\n9-29-2018, version 2:\nMaybe...we may need to do the feature engineering over here...\nLet me save this one as v1, and modify on v2.\n\n9-28-2018, version 1:\n\nAttempt to split the dataset into training and evaluation by using panda.\nGiven the size of data, I'm a bit worried about the time consumption.\n\nGreat reference:\n1. https://www.kaggle.com/szelee/how-to-import-a-csv-file-of-55-million-rows\n\n"

In [2]:
# import sys
# !{sys.executable} -m pip install tqdm

In [3]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
# import matplotlib as plt
import time
from math import sqrt
# import random
# import tensorflow as tf
# import shutil
from sklearn.model_selection import train_test_split



In [4]:
1 + 1

2

In [5]:
def late_night (row):
    if (row['hour'] <= 6) or (row['hour'] >= 20):
        return 1
    else:
        return 0


def night (row):
    if ((row['hour'] <= 20) and (row['hour'] >= 16)) and (row['weekday'] < 5):
        return 1
    else:
        return 0



def clean_df(df):
    df = df.dropna(how = 'any', axis = 'rows')
    df = df[df["fare_amount"] > 2.5]
    df = df[(df["passenger_count"] > 0) & (df["passenger_count"] <= 4)]
    df = df[(df["dropoff_longitude"] > -76) & (df["dropoff_longitude"] < -72)]
    df = df[(df["pickup_longitude"] > -76) & (df["pickup_longitude"] < -72)]
    df = df[(df["pickup_latitude"] > 38) & (df["pickup_latitude"] < 42)]
    df = df[(df["dropoff_latitude"] > 38) & (df["dropoff_latitude"] < 42)]  
    
    df['abs_diff_longitude'] = (df.dropoff_longitude - df.pickup_longitude).abs()
    df['abs_diff_latitude'] = (df.dropoff_latitude - df.pickup_latitude).abs()
    
    df['distance'] = ((df.abs_diff_longitude)**2 + \
                      (df.abs_diff_latitude)**2).apply(lambda x: 69.172*sqrt(x))

    df = df[((df["distance"] <= 25) & (df["fare_amount"] <= 50))\
            | ((df["distance"] > 25) & (df["fare_amount"] >= 50))]
    return df


def add_more_features(df):
    """here.."""
    
#     df['pickup_datetime'] =  pd.to_datetime(df['pickup_datetime'], format='%Y-%m-%d %H:%M:%S %Z') #already done

    df['year'] = df['pickup_datetime'].apply(lambda x: x.year)
    df['month'] = df['pickup_datetime'].apply(lambda x: x.month)
    df['day'] = df['pickup_datetime'].apply(lambda x: x.day)
    df['hour'] = df['pickup_datetime'].apply(lambda x: x.hour)
    df['weekday'] = df['pickup_datetime'].apply(lambda x: x.weekday())
    df['night'] = df.apply (lambda x: night(x), axis=1)
    df['late_night'] = df.apply (lambda x: late_night(x), axis=1)    
    df = df.drop("pickup_datetime",1)
    return df

# Create pandas input function
def make_input_fn(df, num_epochs):
    return tf.estimator.inputs.pandas_input_fn(
    x = add_more_features(df),
    y = df['fare_amount'],
    batch_size = 128*3, # no sure why..
    num_epochs = num_epochs,
    shuffle = True,
    queue_capacity = 1000,
    num_threads = 1
  )

def create_feature_cols():
    return [
    tf.feature_column.numeric_column('passenger_count'),
    tf.feature_column.bucketized_column(tf.feature_column.numeric_column('pickup_longitude'),\
                                        boundaries = np.arange(-76.0, -72, 0.25).tolist()),
    tf.feature_column.bucketized_column(tf.feature_column.numeric_column('dropoff_longitude'),\
                                        boundaries = np.arange(-76.0, -72, 0.25).tolist()),
    tf.feature_column.bucketized_column(tf.feature_column.numeric_column('pickup_latitude'),\
                                        boundaries = np.arange(38.0, 42, 0.25).tolist()), 
    tf.feature_column.bucketized_column(tf.feature_column.numeric_column('dropoff_latitude'),\
                                        boundaries = np.arange(38.0, 42, 0.25).tolist()),         
    tf.feature_column.numeric_column('abs_diff_longitude'),
    tf.feature_column.numeric_column('abs_diff_latitude'),
    tf.feature_column.numeric_column('distance'),
    tf.feature_column.numeric_column('year'),
    tf.feature_column.numeric_column('month'),
    tf.feature_column.numeric_column('day'),
    tf.feature_column.numeric_column('hour'),
    tf.feature_column.numeric_column('weekday'),
    tf.feature_column.numeric_column('night'),
    tf.feature_column.numeric_column('late_night')        
  ]

# Create estimator train and evaluate function
# def train_and_evaluate(output_dir, num_train_steps):
#     estimator = tf.estimator.LinearRegressor(model_dir = output_dir, feature_columns = create_feature_cols())
#     train_spec = tf.estimator.TrainSpec(input_fn = make_input_fn(train_df, 1),\
#                                       max_steps = num_train_steps)
#     eval_spec = tf.estimator.EvalSpec(input_fn = make_input_fn(eval_df, 1), 
#                                     steps = None, 
#                                     start_delay_secs = 1, # start evaluating after N seconds, 
#                                     throttle_secs = 5)  # evaluate every N seconds
#     tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

# np.arange(-76.0, -72, 0.25)


In [6]:
%%time
chunksize = 5_000_000 # alright..with cleaning and featuring.. 5,000,000 might be too large.
chunksize = 2_000_000
traintypes = {
    'key':'str',
    'fare_amount': 'float32',
    'pickup_datetime': 'str', 
    'pickup_longitude': 'float32',
    'pickup_latitude': 'float32',
    'dropoff_longitude': 'float32',
    'dropoff_latitude': 'float32',
    'passenger_count': 'uint8'}

cols = list(traintypes.keys())

df_list = []

CPU times: user 6 µs, sys: 1 µs, total: 7 µs
Wall time: 9.06 µs


In [7]:
# ?tqdm

In [8]:
%%time

dirctory = ".../Kaggle_Taxi/"
print("running on ...", dirctory)
for df_chunk in tqdm(pd.read_csv(dirctory + 'nyc_taxi_train.csv', usecols=cols, dtype=traintypes, chunksize=chunksize)):
    df_chunk['pickup_datetime'] = df_chunk['pickup_datetime'].str.slice(0, 16)
    df_chunk['pickup_datetime'] = pd.to_datetime(df_chunk['pickup_datetime'], utc=True, format='%Y-%m-%d %H:%M')
    
    # Can process each chunk of dataframe here
#     # clean_data(), feature_engineer(),fit()
    df_chunk = clean_df(df_chunk)
    df_chunk = add_more_features(df_chunk)
    print("ok..let's print sth out..might be a bad idea? who knows!")

    tmp_train, tmp_eval = train_test_split(df_chunk, test_size = 0.2)
#     tmp_train.to_csv("chunk_train.csv", index = False, mode = "a")
#     tmp_eval.to_csv("chunk_eval.csv", index = False, mode = "a")    
    tmp_train.to_csv(dirctory + "chunk_train_v2.csv", index = False, mode = "a", header = False)
    tmp_eval.to_csv(dirctory + "chunk_eval_v2.csv", index = False, mode = "a", header = False)
    
#     del tmp_train
#     del tmp_eval
    # Alternatively, append the chunk to list and merge all
    df_list.append(df_chunk)

# data =  pd.read_csv('nyc_taxi_train.csv',usecols=cols, dtype=traintypes)
# data =  pd.read_csv('nyc_taxi_train.csv', nrows = 1000)



0it [00:00, ?it/s]

running on ... /Users/toby/Downloads/Data/Kaggle_Taxi/
ok..let's print sth out..might be a bad idea? who knows!


1it [05:05, 305.14s/it]

KeyboardInterrupt: 

In [9]:
# ? pd.DataFrame.to_csv mode

In [10]:
%%time 

data = pd.concat(df_list)

# Delete the dataframe list to release memory
del df_list

# See what we have loaded
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1742486 entries, 0 to 1999999
Data columns (total 17 columns):
key                   object
fare_amount           float32
pickup_longitude      float32
pickup_latitude       float32
dropoff_longitude     float32
dropoff_latitude      float32
passenger_count       uint8
abs_diff_longitude    float32
abs_diff_latitude     float32
distance              float64
year                  int64
month                 int64
day                   int64
hour                  int64
weekday               int64
night                 int64
late_night            int64
dtypes: float32(7), float64(1), int64(7), object(1), uint8(1)
memory usage: 181.1+ MB
CPU times: user 165 ms, sys: 69.2 ms, total: 234 ms
Wall time: 236 ms


In [11]:
%%time

train_df, eval_df = train_test_split(data, test_size = 0.2)
print(train_df.shape)

train_df.info()

# train_df.to_csv("nyc_taxi_df_train_v2.csv", index = False)
# eval_df.to_csv("nyc_taxi_df_eval_v2.csv", index = False)

(1393988, 17)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1393988 entries, 125677 to 1438048
Data columns (total 17 columns):
key                   1393988 non-null object
fare_amount           1393988 non-null float32
pickup_longitude      1393988 non-null float32
pickup_latitude       1393988 non-null float32
dropoff_longitude     1393988 non-null float32
dropoff_latitude      1393988 non-null float32
passenger_count       1393988 non-null uint8
abs_diff_longitude    1393988 non-null float32
abs_diff_latitude     1393988 non-null float32
distance              1393988 non-null float64
year                  1393988 non-null int64
month                 1393988 non-null int64
day                   1393988 non-null int64
hour                  1393988 non-null int64
weekday               1393988 non-null int64
night                 1393988 non-null int64
late_night            1393988 non-null int64
dtypes: float32(7), float64(1), int64(7), object(1), uint8(1)
memory usage: 144.9+ MB


In [12]:
# ? pd.DataFrame.to_csv

In [13]:
# train_df.head()

In [14]:
# eval_df.head()

In [15]:

os.system("Toby, 'job run finished!'")

32512

In [16]:
%%time
train_df.describe()

CPU times: user 1.64 s, sys: 171 ms, total: 1.81 s
Wall time: 1.84 s


,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,abs_diff_longitude,abs_diff_latitude,distance,year,month,day,hour,weekday,night,late_night
count,1.393988e+06,1.393988e+06,1.393988e+06,1.393988e+06,1.393988e+06,1.393988e+06,1.393988e+06,1.393988e+06,1.393988e+06,1.393988e+06,1.393988e+06,1.393988e+06,1.393988e+06,1.393988e+06,1.393988e+06,1.393988e+06
mean,1.073410e+01,-7.397647e+01,4.075146e+01,-7.397483e+01,4.075176e+01,1.331866e+00,2.095297e-02,2.045492e-02,2.218050e+00,2.011720e+03,6.266836e+00,1.569619e+01,1.351139e+01,3.038997e+00,2.008317e-01,3.702327e-01
std,7.871475e+00,3.548883e-02,2.939705e-02,3.557425e-02,3.313845e-02,6.789118e-01,2.915793e-02,2.115920e-02,2.322281e+00,1.863525e+00,3.436255e+00,8.684633e+00,6.520965e+00,1.949509e+00,4.006226e-01,4.828671e-01
min,2.510000e+00,-7.570316e+01,3.897653e+01,-7.572812e+01,3.895553e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.009000e+03,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,6.000000e+00,-7.399236e+01,4.073687e+01,-7.399158e+01,4.073603e+01,1.000000e+00,6.149292e-03,6.946564e-03,8.869912e-01,2.010000e+03,3.000000e+00,8.000000e+00,9.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
50%,8.500000e+00,-7.398218e+01,4.075351e+01,-7.398066e+01,4.075402e+01,1.000000e+00,1.258850e-02,1.404572e-02,1.496936e+00,2.012000e+03,6.000000e+00,1.600000e+01,1.400000e+01,3.000000e+00,0.000000e+00,0.000000e+00
75%,1.250000e+01,-7.396874e+01,4.076761e+01,-7.396574e+01,4.076845e+01,1.000000e+00,2.339935e-02,2.666092e-02,2.619368e+00,2.013000e+03,9.000000e+00,2.300000e+01,1.900000e+01,5.000000e+00,0.000000e+00,1.000000e+00
max,4.950000e+02,-7.218873e+01,4.191632e+01,-7.215101e+01,4.192382e+01,4.000000e+00,1.618820e+00,8.809624e-01,1.140469e+02,2.015000e+03,1.200000e+01,3.100000e+01,2.300000e+01,6.000000e+00,1.000000e+00,1.000000e+00


In [17]:
# %%time
# tmp = train_df[train_df['fare_amount'].apply(lambda x: type(x)==np.float32)]
# tmp

In [18]:
# ? pd.DataFrame.to_csv

In [19]:
# print(train_df.shape, tmp.shape)

In [20]:
# type(tmp.iloc[0].fare_amount)

In [21]:
# df = pd.read_csv("chunk_eval.csv")


In [22]:
# print(df.shape)
# df.head()

In [23]:
# df.info()

In [24]:
# ##omg... i need to preprocess test df over here...sad!
# test = pd.read_csv('test.csv')
# # test['fare_amount'] = 3
# print('setting fake y for preprocessing purpose..',test.shape, "wait...no I shallnt clean test...")
# # test = clean_df(test)

# def add_more_features_test(df):
#     """here.."""
#     df['abs_diff_longitude'] = (df.dropoff_longitude - df.pickup_longitude).abs()
#     df['abs_diff_latitude'] = (df.dropoff_latitude - df.pickup_latitude).abs()
#     df['distance'] = ((df.abs_diff_longitude)**2 + \
#                       (df.abs_diff_latitude)**2).apply(lambda x: 69.172*sqrt(x))    
#     df['pickup_datetime'] =  pd.to_datetime(df['pickup_datetime'], format='%Y-%m-%d %H:%M:%S %Z')
#     df['year'] = df['pickup_datetime'].apply(lambda x: x.year)
#     df['month'] = df['pickup_datetime'].apply(lambda x: x.month)
#     df['day'] = df['pickup_datetime'].apply(lambda x: x.day)
#     df['hour'] = df['pickup_datetime'].apply(lambda x: x.hour)
#     df['weekday'] = df['pickup_datetime'].apply(lambda x: x.weekday())
#     df['night'] = df.apply (lambda x: night(x), axis=1)
#     df['late_night'] = df.apply (lambda x: late_night(x), axis=1)    
#     df = df.drop("pickup_datetime",1)
#     return df

# print('shape of test after cleaning...',test.shape)
# test = add_more_features_test(test)
# print('shape of test after featuring...',test.shape)

# test.to_csv("featured_test.csv", index = False, mode = "w")

In [25]:
# print(test.shape)
# test